In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm, LinearSegmentedColormap

---

# open data

In [ ]:
# data folder(s) (shall have "/" at the end):
masterpath = "/DATA_MASTER_PATH/"
datapath = [
    masterpath + "23_hike_pinunu-background/2305_zoptical-zanalyze_training/",
    #masterpath + "23_hike_pinunu-background/2305_zoptical-zanalyze_2pi_mass_prod/"
]

# data selection criteria, set here:
def datasel(name):
    sel = (".csv" in name) & ("_V2_" in name) & (not ("lambda" in name))
    for i in range(500):
        if i>100:
            sel = sel &  (not ("_"+str(i)+"." in name))
    return sel

filenames = []
for s in datapath:
    filenames += [s+f.name for f in list(os.scandir(s)) if datasel(f.name)]
print("list of files to open:")
print(*filenames, sep="\n")

In [ ]:
%%time

df = pd.DataFrame()
for name in filenames:
    print("opening file %s" % name.rsplit('/', 1)[-1])
    df0 = pd.read_csv(name)
    df0["file"] = name.replace(".csv", "")
    df = df.append(df0, ignore_index=True, sort=False)
    
print("total events: %d" % df.shape[0])
for i_class in df["class"].unique():
    print("events in class %d: %d" % (i_class, df[df["class"]==i_class].shape[0]))
print("---")

In [ ]:
# there are faulty vertex (as reconstructed by the calorimeter) data --> throwing them away
if True:
    print("removing %d broken-vertex events (%f%%)..." % (
        df[df.Vertex_xRec_Z.isnull()].shape[0], df[df.Vertex_xRec_Z.isnull()].shape[0]/df.shape[0]*100
    ))
    df = df[~df.Vertex_xRec_Z.isnull()]

In [ ]:
nEvsPop = None  # (max.) nr. of events per class

if not (nEvsPop is None):
    print("after descaling (w/ %d events per class):" % nEvsPop)

    classdf = []
    for i, i_class in enumerate(df["class"].unique()):
        classdf.append(df[df["class"]==i_class].sample(frac=1).head(nEvsPop))
        print("events in class %d: %d" % (i_class, classdf[i].shape[0]))

    df = pd.concat(classdf)
    print("total events: %d" % df.shape[0])

In [ ]:
print("list of variables (%d):" % df.shape[1])
print(df.columns)

---

# condition datasets

In [ ]:
# class index

# set all possible labels here:
classlabel = {
    "k-2pi" : 0,
    "k-pinunu" : 1,
    "lambda-pin" : 5,
}

for i_class in df["class"].unique():
    print("class = %d <--> filename type is e.g. %s" % (i_class, str(df[df["class"] == i_class].file.head(1).values[0]).rsplit('/', 1)[-1]))

In [ ]:
# variable conditioning

# select best vertex quantities
df.loc[df.Vertex_nConverted==0, "Vertex_xBest_Z"] = df.Vertex_xRec_Z
df.loc[df.Vertex_nConverted!=0, "Vertex_xBest_Z"] = df.Vertex_xRecPre_Z
df.loc[df.Vertex_nConverted==0, "Vertex_xBest_X"] = df.Vertex_xRec_X
df.loc[df.Vertex_nConverted!=0, "Vertex_xBest_X"] = df.Vertex_xRecPre_X
df.loc[df.Vertex_nConverted==0, "Vertex_xBest_Y"] = df.Vertex_xRec_Y
df.loc[df.Vertex_nConverted!=0, "Vertex_xBest_Y"] = df.Vertex_xRecPre_Y
df.loc[df.Vertex_nConverted==0, "Vertex_pBest0_Z"] = df.Vertex_pRec0_Z
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest0_Z"] = df.Vertex_pRecPre0_Z
df.loc[df.Vertex_nConverted==0, "Vertex_pBest0_X"] = df.Vertex_pRec0_X
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest0_X"] = df.Vertex_pRecPre0_X
df.loc[df.Vertex_nConverted==0, "Vertex_pBest0_Y"] = df.Vertex_pRec0_Y
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest0_Y"] = df.Vertex_pRecPre0_Y
df.loc[df.Vertex_nConverted==0, "Vertex_pBest1_Z"] = df.Vertex_pRec1_Z
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest1_Z"] = df.Vertex_pRecPre1_Z
df.loc[df.Vertex_nConverted==0, "Vertex_pBest1_X"] = df.Vertex_pRec1_X
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest1_X"] = df.Vertex_pRecPre1_X
df.loc[df.Vertex_nConverted==0, "Vertex_pBest1_Y"] = df.Vertex_pRec1_Y
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest1_Y"] = df.Vertex_pRecPre1_Y

# compute all transverse quantities from cartesian components
df["Vertex_xRec_T"] = np.sqrt( df.Vertex_xRec_X**2 + df.Vertex_xRec_Y**2 )
df["Vertex_xRecPre_T"] = np.sqrt( df.Vertex_xRecPre_X**2 + df.Vertex_xRecPre_Y**2 )
df["Cluster0_xRec_T"] = np.sqrt( df.Cluster0_xRec_X**2 + df.Cluster0_xRec_Y**2 )
df["Cluster1_xRec_T"] = np.sqrt( df.Cluster1_xRec_X**2 + df.Cluster1_xRec_Y**2 )
df["Vertex_pRec0_T"] = np.sqrt( df.Vertex_pRec0_X**2 + df.Vertex_pRec0_Y**2 )
df["Vertex_pRec1_T"] = np.sqrt( df.Vertex_pRec1_X**2 + df.Vertex_pRec1_Y**2 )
df["Cluster0_xPre1_T"] = np.sqrt( df.Cluster0_xPre1_X**2 + df.Cluster0_xPre1_Y**2 )
df["Cluster0_xPre2_T"] = np.sqrt( df.Cluster0_xPre2_X**2 + df.Cluster0_xPre2_Y**2 )
df["Cluster1_xPre1_T"] = np.sqrt( df.Cluster1_xPre1_X**2 + df.Cluster1_xPre1_Y**2 )
df["Cluster1_xPre2_T"] = np.sqrt( df.Cluster1_xPre2_X**2 + df.Cluster1_xPre2_Y**2 )
df["Vertex_pRecPre0_T"] = np.sqrt( df.Vertex_pRecPre0_X**2 + df.Vertex_pRecPre0_Y**2 )
df["Vertex_pRecPre1_T"] = np.sqrt( df.Vertex_pRecPre1_X**2 + df.Vertex_pRecPre1_Y**2 )

df["Vertex_xBest_T"] = np.sqrt( df.Vertex_xBest_X**2 + df.Vertex_xBest_Y**2 )
df["Vertex_pBest0_T"] = np.sqrt( df.Vertex_pBest0_X**2 + df.Vertex_pBest0_Y**2 )
df["Vertex_pBest1_T"] = np.sqrt( df.Vertex_pBest1_X**2 + df.Vertex_pBest1_Y**2 )

# compute new 2-cluster position and momentum-related variables
df["Clusters_xRec_TMin"] = df[["Cluster0_xRec_T", "Cluster1_xRec_T"]].min(axis=1)
df["Clusters_xPre1_TMin"] = df[["Cluster0_xPre1_T", "Cluster1_xPre1_T"]].min(axis=1)
df["Clusters_xPre2_TMin"] = df[["Cluster0_xPre2_T", "Cluster1_xPre2_T"]].min(axis=1)
df["Vertex_pRec_TMin"] = df[["Vertex_pRec0_T", "Vertex_pRec1_T"]].min(axis=1)
df["Vertex_pRecPre_TMin"] = df[["Vertex_pRecPre0_T", "Vertex_pRecPre1_T"]].min(axis=1)
df["Clusters_xRec_TMax"] = df[["Cluster0_xRec_T", "Cluster1_xRec_T"]].max(axis=1)
df["Clusters_xPre1_TMax"] = df[["Cluster0_xPre1_T", "Cluster1_xPre1_T"]].max(axis=1)
df["Clusters_xPre2_TMax"] = df[["Cluster0_xPre2_T", "Cluster1_xPre2_T"]].max(axis=1)
df["Vertex_pRec_TMax"] = df[["Vertex_pRec0_T", "Vertex_pRec1_T"]].max(axis=1)
df["Vertex_pRecPre_TMax"] = df[["Vertex_pRecPre0_T", "Vertex_pRecPre1_T"]].max(axis=1)
df["Clusters_xRec_TSum"] = df.Clusters_xRec_TMax + df.Clusters_xRec_TMin
df["Clusters_xPre1_TSum"] = df.Clusters_xPre1_TMax + df.Clusters_xPre1_TMin
df["Clusters_xPre2_TSum"] = df.Clusters_xPre2_TMax + df.Clusters_xPre2_TMin
df["Vertex_pRec_TSum"] = df.Vertex_pRec_TMax + df.Vertex_pRec_TMin
df["Vertex_pRecPre_TSum"] = df.Vertex_pRecPre_TMax + df.Vertex_pRecPre_TMin
df["Clusters_xRec_TDif"] = df.Clusters_xRec_TMax - df.Clusters_xRec_TMin
df["Clusters_xPre1_TDif"] = df.Clusters_xPre1_TMax - df.Clusters_xPre1_TMin
df["Clusters_xPre2_TDif"] = df.Clusters_xPre2_TMax - df.Clusters_xPre2_TMin
df["Vertex_pRec_TDif"] = df.Vertex_pRec_TMax - df.Vertex_pRec_TMin
df["Vertex_pRecPre_TDif"] = df.Vertex_pRecPre_TMax - df.Vertex_pRecPre_TMin
df["Clusters_xRec_TAsym"] = df.Clusters_xRec_TDif / df.Clusters_xRec_TSum
df["Clusters_xPre1_TAsym"] = df.Clusters_xPre1_TDif / df.Clusters_xPre1_TSum
df["Clusters_xPre2_TAsym"] = df.Clusters_xPre2_TDif / df.Clusters_xPre2_TSum
df["Vertex_pRec_TAsym"] = df.Vertex_pRec_TDif / df.Vertex_pRec_TSum
df["Vertex_pRecPre_TAsym"] = df.Vertex_pRecPre_TDif / df.Vertex_pRecPre_TSum

df["Vertex_pBest_TMin"] = df[["Vertex_pBest0_T", "Vertex_pBest1_T"]].min(axis=1)
df["Vertex_pBest_TMax"] = df[["Vertex_pBest0_T", "Vertex_pBest1_T"]].max(axis=1)
df["Vertex_pBest_TSum"] = df.Vertex_pBest_TMax + df.Vertex_pBest_TMin
df["Vertex_pBest_TDif"] = df.Vertex_pBest_TMax - df.Vertex_pBest_TMin
df["Vertex_pBest_TAsym"] = df.Vertex_pBest_TDif / df.Vertex_pBest_TSum

df["Clusters_xDist"] = np.sqrt(  # distance between clusters
    (df.Cluster1_xRec_X-df.Cluster0_xRec_X)**2 + (df.Cluster1_xRec_Y-df.Cluster0_xRec_Y)**2
)

# compute new 2-cluster energy-related variables
df["Clusters_EMin"] = df[["Cluster0_ERec", "Cluster1_ERec"]].min(axis=1)
df["Clusters_EMax"] = df[["Cluster0_ERec", "Cluster1_ERec"]].max(axis=1)
df["Clusters_ESum"] = df.Clusters_EMax + df.Clusters_EMin
df["Clusters_EDif"] = df.Clusters_EMax - df.Clusters_EMin
df["Clusters_EAsym"] = df.Clusters_EDif / df.Clusters_ESum

df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Clusters_xRec_TEMin"] = df.Cluster0_xRec_T
df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Clusters_xRec_TEMax"] = df.Cluster1_xRec_T
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Clusters_xRec_TEMin"] = df.Cluster1_xRec_T
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Clusters_xRec_TEMax"] = df.Cluster0_xRec_T

df["Clusters_ECOG"] = np.sqrt(  # centre of gravity between clusters (weighted with energies)
    (
        (df.Cluster0_ERec*df.Cluster0_xRec_X + df.Cluster1_ERec*df.Cluster1_xRec_X)**2 +\
        (df.Cluster0_ERec*df.Cluster0_xRec_Y + df.Cluster1_ERec*df.Cluster1_xRec_Y)**2
    ) / df.Clusters_ESum**2
)

# compute pion kinematics from two photons (pion energy is simply Clusters_ESum)
df["Vertex_pRecPi_Z"] = df.Vertex_pRec0_Z + df.Vertex_pRec1_Z
df["Vertex_pRecPi_X"] = df.Vertex_pRec0_X + df.Vertex_pRec1_X
df["Vertex_pRecPi_Y"] = df.Vertex_pRec0_Y + df.Vertex_pRec1_Y
df["Vertex_pRecPrePi_Z"] = df.Vertex_pRecPre0_Z + df.Vertex_pRecPre1_Z
df["Vertex_pRecPrePi_X"] = df.Vertex_pRecPre0_X + df.Vertex_pRecPre1_X
df["Vertex_pRecPrePi_Y"] = df.Vertex_pRecPre0_Y + df.Vertex_pRecPre1_Y

df["Vertex_pRecPi_T"] = np.sqrt( df.Vertex_pRecPi_X**2 + df.Vertex_pRecPi_Y**2 )
df["Vertex_pRecPrePi_T"] = np.sqrt( df.Vertex_pRecPrePi_X**2 + df.Vertex_pRecPrePi_Y**2 )

df["Vertex_pRecPi_sTh"] = df.Vertex_pRecPi_T / np.sqrt(  # pion propagation angle wrt. beam
    df.Vertex_pRecPi_T**2 + df.Vertex_pRecPi_Z**2
)
df["Vertex_pRecPrePi_sTh"] = df.Vertex_pRecPrePi_T / np.sqrt(  # pion propagation angle wrt. beam
    df.Vertex_pRecPrePi_T**2 + df.Vertex_pRecPrePi_Z**2
)

df["Vertex_pBestPi_Z"] = df.Vertex_pBest0_Z + df.Vertex_pBest1_Z
df["Vertex_pBestPi_X"] = df.Vertex_pBest0_X + df.Vertex_pBest1_X
df["Vertex_pBestPi_Y"] = df.Vertex_pBest0_Y + df.Vertex_pBest1_Y

df["Vertex_pBestPi_T"] = np.sqrt( df.Vertex_pBestPi_X**2 + df.Vertex_pBestPi_Y**2 )

df["Vertex_pBestPi_sTh"] = df.Vertex_pBestPi_T / np.sqrt(  # pion propagation angle wrt. beam
    df.Vertex_pBestPi_T**2 + df.Vertex_pBestPi_Z**2
)

print("(preliminary) nr. of classification variables before drop: %d" % df.shape[1])

# drop everything that isn't needed anymore
df = df.drop(columns=[
    
    # raw variables
    "Vertex_xRec_X", "Vertex_xRec_Y",
    "Vertex_xRecPre_X", "Vertex_xRecPre_Y",
    "Cluster0_xRec_X", "Cluster0_xRec_Y",
    "Cluster1_xRec_X", "Cluster1_xRec_Y",
    "Vertex_pRec0_X", "Vertex_pRec0_Y",
    "Vertex_pRec1_X", "Vertex_pRec1_Y",
    "Cluster0_xPre1_X", "Cluster0_xPre1_Y",
    "Cluster0_xPre2_X", "Cluster0_xPre2_Y",
    "Cluster1_xPre1_X", "Cluster1_xPre1_Y",
    "Cluster1_xPre2_X", "Cluster1_xPre2_Y",
    "Vertex_pRecPre0_X", "Vertex_pRecPre0_Y",
    "Vertex_pRecPre1_X", "Vertex_pRecPre1_Y",
    
    # newly created variables
    "Vertex_xBest_X", "Vertex_xBest_Y",
    "Vertex_pBest0_X", "Vertex_pBest0_Y",
    "Vertex_pBest1_X", "Vertex_pBest1_Y",
    
    "Vertex_pRecPi_X", "Vertex_pRecPi_Y",
    "Vertex_pRecPrePi_X", "Vertex_pRecPrePi_Y",
])

print("(preliminary) nr. of classification variables after drop: %d" % df.shape[1])
print("---")

---

# boolean analysis

In [ ]:
bPreBool = True  # perform preliminary boolean analysis, to be compare to the results in the LoI?

shift_fv = 150  # FV frame shift to match the data reference system

In [ ]:
if bPreBool:

    # booleans
    bool_sig = df["class"] == classlabel["k-pinunu"]
    bool_bkg = ~bool_sig
    bool_2mec = df["class"] != -9999
    bool_fv = (df.Vertex_xRec_Z > (280-shift_fv)) & (df.Vertex_xRec_Z < (350-shift_fv))
    bool_rmin = df.Clusters_xRec_TMin>35e-2
    bool_emin = df.Clusters_EMin > (2 / (df.Clusters_xRec_TEMin))
    bool_pt = df.Vertex_pRecPi_T > 0.140
    bool_psv = (df.Vertex_nConverted > 0) & (df.Vertex_xRecPre_Z < (350-shift_fv)) & (df.Vertex_pRecPrePi_T > 0.140)
    bool_even = df.iPair == 1
    bool_nofused = (df.Cluster0_nHits == 1) & (df.Cluster1_nHits == 1)

    # now count events...

    # with 2 photons in MEC (i.e. the total dataset for each class)
    n_sig_2mec = df[bool_sig & bool_2mec].shape[0]
    n_bkg_2mec = df[bool_bkg & bool_2mec].shape[0]
    n_bkgEv_2mec = df[bool_nofused & bool_even & bool_bkg & bool_2mec].shape[0]
    n_bkgOd_2mec = df[bool_nofused & ~bool_even & bool_bkg & bool_2mec].shape[0]
    n_bkgFs_2mec = df[~bool_nofused & bool_bkg & bool_2mec].shape[0]

    # reconstructed in FV with MEC only
    n_sig_2mec_fv = df[bool_sig & bool_2mec & bool_fv].shape[0]
    n_bkg_2mec_fv = df[bool_bkg & bool_2mec & bool_fv].shape[0]
    n_bkgEv_2mec_fv = df[bool_nofused & bool_even & bool_bkg & bool_2mec & bool_fv].shape[0]
    n_bkgOd_2mec_fv = df[bool_nofused & ~bool_even & bool_bkg & bool_2mec & bool_fv].shape[0]
    n_bkgFs_2mec_fv = df[~bool_nofused & bool_bkg & bool_2mec & bool_fv].shape[0]

    # cut on minimum-radius cluster (>35 cm)
    n_sig_2mec_fv_rmin = df[bool_sig & bool_2mec & bool_fv & bool_rmin].shape[0]
    n_bkg_2mec_fv_rmin = df[bool_bkg & bool_2mec & bool_fv & bool_rmin].shape[0]
    n_bkgEv_2mec_fv_rmin = df[bool_nofused & bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin].shape[0]
    n_bkgOd_2mec_fv_rmin = df[bool_nofused & ~bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin].shape[0]
    n_bkgFs_2mec_fv_rmin = df[~bool_nofused & bool_bkg & bool_2mec & bool_fv & bool_rmin].shape[0]

    # cut minimum-energy cluster (>2 GeV divided by corresponding cluster radius)
    n_sig_2mec_fv_rmin_emin = df[bool_sig & bool_2mec & bool_fv & bool_rmin & bool_emin].shape[0]
    n_bkg_2mec_fv_rmin_emin = df[bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin].shape[0]
    n_bkgEv_2mec_fv_rmin_emin = df[bool_nofused & bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin].shape[0]
    n_bkgOd_2mec_fv_rmin_emin = df[bool_nofused & ~bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin].shape[0]
    n_bkgFs_2mec_fv_rmin_emin = df[~bool_nofused & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin].shape[0]

    # cut on pion transverse momentum (>0.140 GeV) computed with MEC only
    n_sig_2mec_fv_rmin_emin_pt = df[bool_sig & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt].shape[0]
    n_bkg_2mec_fv_rmin_emin_pt = df[bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt].shape[0]
    n_bkgEv_2mec_fv_rmin_emin_pt = df[bool_nofused & bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt].shape[0]
    n_bkgOd_2mec_fv_rmin_emin_pt = df[bool_nofused & ~bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt].shape[0]
    n_bkgFs_2mec_fv_rmin_emin_pt = df[~bool_nofused & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt].shape[0]
    
    # adding the PSV data
    n_sig_2mec_fv_rmin_emin_pt_psv = df[bool_sig & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt & bool_psv].shape[0]
    n_bkg_2mec_fv_rmin_emin_pt_psv = df[bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt & bool_psv].shape[0] 
    n_bkgEv_2mec_fv_rmin_emin_pt_psv = df[bool_nofused & bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt & bool_psv].shape[0] 
    n_bkgOd_2mec_fv_rmin_emin_pt_psv = df[bool_nofused & ~bool_even & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt & bool_psv].shape[0] 
    n_bkgFs_2mec_fv_rmin_emin_pt_psv = df[~bool_nofused & bool_bkg & bool_2mec & bool_fv & bool_rmin & bool_emin & bool_pt & bool_psv].shape[0]     

In [ ]:
if bPreBool:
    
    # total events (with 2 photons in MEC) in the LoI:
    normSig = 302
    normBkg = 9.45e7

    print("counts for signal:")
    print(normSig*n_sig_2mec/n_sig_2mec)
    print(normSig*n_sig_2mec_fv/n_sig_2mec)
    print(normSig*n_sig_2mec_fv_rmin/n_sig_2mec)
    print(normSig*n_sig_2mec_fv_rmin_emin/n_sig_2mec)
    print(normSig*n_sig_2mec_fv_rmin_emin_pt/n_sig_2mec)
    print(normSig*n_sig_2mec_fv_rmin_emin_pt_psv/n_bkg_2mec)

    print("---\ncounts for background (total):")
    print(normBkg*n_bkg_2mec/n_bkg_2mec)
    print(normBkg*n_bkg_2mec_fv/n_bkg_2mec)
    print(normBkg*n_bkg_2mec_fv_rmin_emin/n_bkg_2mec)
    print(normBkg*n_bkg_2mec_fv_rmin_emin_pt/n_bkg_2mec)
    print(normBkg*n_bkg_2mec_fv_rmin_emin_pt_psv/n_bkg_2mec)
    
    print("---\ncounts for background (even, not fused):")
    print(normBkg*n_bkgEv_2mec/n_bkg_2mec)
    print(normBkg*n_bkgEv_2mec_fv/n_bkg_2mec)
    print(normBkg*n_bkgEv_2mec_fv_rmin_emin/n_bkg_2mec)
    print(normBkg*n_bkgEv_2mec_fv_rmin_emin_pt/n_bkg_2mec)
    print(normBkg*n_bkgEv_2mec_fv_rmin_emin_pt_psv/n_bkg_2mec)
    
    print("---\ncounts for background (odd, not fused):")
    print(normBkg*n_bkgOd_2mec/n_bkg_2mec)
    print(normBkg*n_bkgOd_2mec_fv/n_bkg_2mec)
    print(normBkg*n_bkgOd_2mec_fv_rmin_emin/n_bkg_2mec)
    print(normBkg*n_bkgOd_2mec_fv_rmin_emin_pt/n_bkg_2mec)
    print(normBkg*n_bkgOd_2mec_fv_rmin_emin_pt_psv/n_bkg_2mec)
    
    print("---\ncounts for background (fused):")
    print(normBkg*n_bkgFs_2mec/n_bkg_2mec)
    print(normBkg*n_bkgFs_2mec_fv/n_bkg_2mec)
    print(normBkg*n_bkgFs_2mec_fv_rmin_emin/n_bkg_2mec)
    print(normBkg*n_bkgFs_2mec_fv_rmin_emin_pt/n_bkg_2mec)
    print(normBkg*n_bkgFs_2mec_fv_rmin_emin_pt_psv/n_bkg_2mec)

In [ ]:
if bPreBool:
    
    bPreBool_save = True  # save plots?
    
    bLog = True
    cmap_name = "jet"
    range_plot = ((150-shift_fv, 400-shift_fv), (0, 0.4)) 
    
    #cmap = LinearSegmentedColormap.from_list(
    #    "%s_white" % cmap_name, 
    #    list(np.concatenate((np.array([[0, 0, 0, 0]]), plt.get_cmap(cmap_name)(np.arange(256))))),
    #)
    cmap = plt.get_cmap(cmap_name).copy()
    cmap.set_bad('white')
    
    for i_class in (0, 1, 2, 3):
        fig, ax = plt.subplots(num=i_class, nrows=1, ncols=2, figsize=(12, 4))
        
        bool_plot = bool_2mec & bool_rmin & bool_emin
        if i_class==0:
            bool_class = bool_sig
            bool_plot = bool_plot & bool_psv
        elif i_class==1:
            bool_class = bool_bkg & bool_nofused & bool_even 
            bool_plot = bool_plot & bool_nofused & bool_even 
        elif i_class==2:
            bool_class = bool_bkg & bool_nofused & ~bool_even 
            bool_plot = bool_plot & bool_nofused & ~bool_even 
        else:
            bool_class = bool_bkg & ~bool_nofused
            bool_plot = bool_plot & ~bool_nofused
            
        ax[0].hist2d(
            df[bool_class].Vertex_xRec_Z, df[bool_class].Vertex_pRecPi_T,
            bins=(200, 100), range=range_plot, cmap=cmap, norm=LogNorm() if bLog else None, cmin=1
        )
        ax[1].hist2d(
            df[bool_class & bool_plot].Vertex_xRec_Z, df[bool_class & bool_plot].Vertex_pRecPi_T,
            bins=(200, 100), range=range_plot, cmap=cmap, norm=LogNorm() if bLog else None, cmin=1
        )
        
        box = ((280-shift_fv, 1), (280-shift_fv, 0.14), (350-shift_fv, 0.14), (350-shift_fv, 1))
        x_box, y_box = zip(*box)
        ax[0].plot(x_box, y_box, color="0.7", lw=3)
        ax[1].plot(x_box, y_box, color="0.7", lw=3)
        
        titles = ["signal", "background, even, not fused", "background, odd, not fused", "background, fused"]
        fig.suptitle(titles[i_class])
        fig.supxlabel("Vertex_xRec_Z [m]")
        fig.supylabel("Vertex_pRecPi_T [GeV]")
        fig.tight_layout()
        if bPreBool_save:
            fig.savefig("./output_misc/%s.png" % titles[i_class].replace(",", "_").replace(" ", ""))